In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense,Dropout,Flatten, LSTM, BatchNormalization

def crnn_model():
    model = Sequential()

    model.add(Conv1D(256, 3, activation='relu', input_shape=(44100,1)))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.25))

    model.add(LSTM(32))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'])

    return model

print('loaded')

loaded


In [2]:
import pandas as pd
import AudioUtil
import numpy as np
from pathlib import Path
import natsort
from random import randint

def random_exclude(start, end, exclude):
    randInt = randint(start, end)    
    return random_exclude(start, end, exclude) if randInt == exclude else randInt
print('loaded')

loaded


In [3]:
#variables
songs = ['burble','Century','cherry','CopperCoins','Generator','November','Runner','TwirlingTwins']

samplesDir = 'data/samples_3sec/'

listAllTrainingsPairs = []
listAllTrainingsLabel = []
print('loaded')

loaded


In [5]:
import pandas as pd
import AudioUtil
import numpy as np
from pathlib import Path
import natsort
from random import randint

#%%
#song burble

#generate list with all paths to samples
for song in songs:
    print('Loading Song: ' + song)
    pathListSong = []
    for file in Path(samplesDir + song).glob('*.wav'):
        #print('Found Path: ' + file.name)
        pathListSong.append(file.name)

    pathListSong = natsort.natsorted(pathListSong)
    for idx, s in enumerate(pathListSong):
        pathListSong[idx] = samplesDir + song + '/' + s
        
    for idx, filePath in enumerate(pathListSong):
        sample, _ = AudioUtil.loadWaveform(filePath)
        
        if idx+1 < len(pathListSong):
            sampleMatch, _ = AudioUtil.loadWaveform(pathListSong[idx+1])
            match = np.concatenate((sample, sampleMatch))
            match = match[44099:88199]
            listAllTrainingsPairs.append(match.astype(np.float32))
            listAllTrainingsLabel.append(1)
            
            listAllTrainingsPairs.append(match.astype(np.float32))
            listAllTrainingsLabel.append(1)
            
            listAllTrainingsPairs.append(match.astype(np.float32))
            listAllTrainingsLabel.append(1)
            
            listAllTrainingsPairs.append(match.astype(np.float32))
            listAllTrainingsLabel.append(1)
            
            #listAllTrainingsPairs.append(match)
            #listAllTrainingsLabel.append(1)
        
        if idx+2 < len(pathListSong):
            sampleWrongMatch, _ = AudioUtil.loadWaveform(pathListSong[idx+2])
            noMatch = np.concatenate((sample, sampleWrongMatch))
            noMatch = noMatch[44099:88199]
            listAllTrainingsPairs.append(noMatch.astype(np.float32))
            listAllTrainingsLabel.append(0)
        else:
            sampleWrongMatch, _ = AudioUtil.loadWaveform(pathListSong[idx-1])
            noMatch = np.concatenate((sample, sampleWrongMatch))
            noMatch = noMatch[44099:88199]
            listAllTrainingsPairs.append(noMatch.astype(np.float32))
            listAllTrainingsLabel.append(0)

        wrongSample1, _ = AudioUtil.loadWaveform(pathListSong[random_exclude(0, len(pathListSong)-1, idx+1)])
        noMatch1 = np.concatenate((sample, wrongSample1))
        noMatch1 = noMatch1[44099:88199]
        listAllTrainingsPairs.append(noMatch1.astype(np.float32))
        listAllTrainingsLabel.append(0)
        
        wrongSample2, _ = AudioUtil.loadWaveform(pathListSong[random_exclude(0, len(pathListSong)-1, idx+1)])
        noMatch2 = np.concatenate((sample, wrongSample2))
        noMatch2 = noMatch2[44099:88199]
        listAllTrainingsPairs.append(noMatch2.astype(np.float32))
        listAllTrainingsLabel.append(0)
        
        wrongSample3, _ = AudioUtil.loadWaveform(pathListSong[random_exclude(0, len(pathListSong)-1, idx+1)])
        noMatch3 = np.concatenate((sample, wrongSample3))
        noMatch3 = noMatch3[44099:88199]
        listAllTrainingsPairs.append(noMatch3.astype(np.float32))
        listAllTrainingsLabel.append(0)

        wrongSample4, _ = AudioUtil.loadWaveform(pathListSong[random_exclude(0, len(pathListSong)-1, idx+1)])
        noMatch4 = np.concatenate((sample, wrongSample4))
        noMatch4 = noMatch4[44099:88199]
        listAllTrainingsPairs.append(noMatch4.astype(np.float32))
        listAllTrainingsLabel.append(0)

        wrongSample5, _ = AudioUtil.loadWaveform(pathListSong[random_exclude(0, len(pathListSong)-1, idx+1)])
        noMatch5 = np.concatenate((sample, wrongSample5))
        noMatch5 = noMatch5[44099:88199]
        listAllTrainingsPairs.append(noMatch5.astype(np.float32))
        listAllTrainingsLabel.append(0)

print('Count Samplepairs: ' + str(len(listAllTrainingsPairs)))
print('Count Labels: ' + str(len(listAllTrainingsLabel)))

Loading Song: burble
Loading Song: Century
Loading Song: cherry
Loading Song: CopperCoins
Loading Song: Generator
Loading Song: November
Loading Song: Runner
Loading Song: TwirlingTwins
Count Samplepairs: 5848
Count Labels: 5848


In [ ]:
print(listAllTrainingsPairs[0])
print(len(listAllTrainingsPairs[0]))

In [6]:
#shuffle and split data into training and validation
print('split training/ test')
from sklearn.model_selection import train_test_split

df_train = pd.DataFrame({'train':listAllTrainingsPairs, 'label':listAllTrainingsLabel})

#X_train, X_test, y_train, y_test = train_test_split(listAllTrainingsPairs, listAllTrainingsLabel, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(df_train['train'], df_train['label'], test_size=0.3)

X_train = np.stack(X_train.values).astype(np.float32)
y_train = np.stack(y_train.values).astype(int)
y_train = np.reshape(y_train, (-1, 1))

split training/ test


In [4]:
#compile model
print('compile')
#model = cRNN_Prototyp()
#model = tf.saved_model.load('SL_cRNN_prototyp_train_all_2')
#model = keras.models.load_model('model_crnn.h5')
model = crnn_model()
model.summary()

compile


2022-06-26 20:22:45.579038: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-26 20:22:46.169577: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11411 MB memory:  -> device: 0, name: TITAN Xp, pci bus id: 0000:09:00.0, compute capability: 6.1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 44098, 256)        1024      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 22049, 256)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22049, 256)        0         
                                                                 
 lstm (LSTM)                 (None, 32)                36992     
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                        

In [7]:
#train and save model
batch = 16
epoch = 75
model_name = 'crnn_model_final.h5'

model.fit(X_train, y_train, batch, epochs=epoch,
                      validation_split=0.1, shuffle=True)
model.save(model_name)

Epoch 1/75


2022-06-26 20:34:02.344923: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


231/231 [==============================] - 102s 429ms/step - loss: 0.6775 - accuracy: 0.6022 - val_loss: 0.6814 - val_accuracy: 0.5756
Epoch 2/75
231/231 [==============================] - 99s 429ms/step - loss: 0.6739 - accuracy: 0.6049 - val_loss: 0.6835 - val_accuracy: 0.5756
Epoch 3/75
231/231 [==============================] - 98s 426ms/step - loss: 0.6737 - accuracy: 0.6049 - val_loss: 0.6813 - val_accuracy: 0.5756
Epoch 4/75
231/231 [==============================] - 99s 428ms/step - loss: 0.6745 - accuracy: 0.6047 - val_loss: 0.6819 - val_accuracy: 0.5756
Epoch 5/75
231/231 [==============================] - 99s 430ms/step - loss: 0.6750 - accuracy: 0.6049 - val_loss: 0.6813 - val_accuracy: 0.5756
Epoch 6/75
231/231 [==============================] - 100s 431ms/step - loss: 0.6728 - accuracy: 0.6052 - val_loss: 0.6815 - val_accuracy: 0.5756
Epoch 7/75
231/231 [==============================] - 100s 431ms/step - loss: 0.6729 - accuracy: 0.6049 - val_loss: 0.6810 - val_accuracy: 